# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key
#api_key = weather_api_key
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [38]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)
print(lat_lngs)
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
#debugging no more than 60 per min
cities=cities[0:50]
cities

['anta',
 'fuzhou',
 'bluff',
 'lebu',
 'waipawa',
 'rikitea',
 'belyy yar',
 'port macquarie',
 'nikolskoye',
 'sofiysk',
 'tumannyy',
 'cape town',
 'mataura',
 'hashtgerd',
 'bredasdorp',
 'port elizabeth',
 'albany',
 'ostrovnoy',
 'qaanaaq',
 'tasiilaq',
 'hilo',
 'jamestown',
 'akdagmadeni',
 'faanui',
 'eyl',
 'san patricio',
 'busselton',
 'provideniya',
 'punta arenas',
 'swinoujscie',
 'khatanga',
 'vardo',
 'ushuaia',
 'maldonado',
 'orocue',
 'puerto ayora',
 'atuona',
 'virginia beach',
 'batagay',
 'coruripe',
 'butaritari',
 'panguna',
 'barentsburg',
 'labutta',
 'kodiak',
 'georgetown',
 'castro',
 'narsaq',
 'taman',
 'windhoek']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [39]:
# Perform a weather check on each city using a series of successive API calls.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

weather_url = f"{url}appid={weather_api_key}&units={units}"
weather_url


'http://api.openweathermap.org/data/2.5/weather?appid=775126892489447e8d019010166b439e&units=Imperial'

In [50]:
city_data_list = []

#number
number=0
series_number = 1
# Include a print log of each city as it'sbeing processed (with the city number and city name).
for city in cities:
    city_dict = {}
    try:
        query_url = f'{weather_url}&q={city}'
        
        response = requests.get(query_url).json()
        
        city_dict['city'] = city
        city_dict['temp_max']= response['main']['temp_max']
        city_dict['humidity']= response['main']['humidity']
        city_dict['clouds']= response['clouds']['all']
        city_dict['windspeed']= response['wind']['speed']
        city_dict['lat']=response['coord']['lat']
        city_dict['lng']= response['coord']['lon']
        city_dict['country']= response['sys']['country']
        city_dict['date']= (response['dt'])
        #append to list
        city_data_list.append(city_dict)
        if number % 50:
            time.sleep(60)
#             number = 1
#             series_number += 1
#             city_data_list(city_dict)
#         else:
#             number += 1
#             city_data_list(city)
        print("- Processing next 50 -")
        print("---------------------")
        print(f'{city} is {number} of {len(cities)}')
    except:
        print('City not found. Skipping...')
        pass
        
print("-----------------------")       
print("Processing Finished!")

- Processing request -
---------------------
anta is 0 of 50
- Processing request -
---------------------
fuzhou is 0 of 50
- Processing request -
---------------------
bluff is 0 of 50
- Processing request -
---------------------
lebu is 0 of 50
- Processing request -
---------------------
waipawa is 0 of 50
- Processing request -
---------------------
rikitea is 0 of 50
- Processing request -
---------------------
belyy yar is 0 of 50
- Processing request -
---------------------
port macquarie is 0 of 50
- Processing request -
---------------------
nikolskoye is 0 of 50
City not found. Skipping...
City not found. Skipping...
- Processing request -
---------------------
cape town is 0 of 50
- Processing request -
---------------------
mataura is 0 of 50
- Processing request -
---------------------
hashtgerd is 0 of 50
- Processing request -
---------------------
bredasdorp is 0 of 50
- Processing request -
---------------------
port elizabeth is 0 of 50
- Processing request -
--------

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [54]:
# Output File (CSV)
output_data_file = "output_data/cities.csv"
#convert to dataframe
weather_data = pd.DataFrame(city_data_list)
weather_data.to_csv(output_data_file)
weather_data.head()

,city,temp_max,humidity,clouds,windspeed,lat,lng,country,date
0,anta,60.17,33,0,1.23,25.1500,76.3000,IN,1612719352
1,fuzhou,62.60,72,75,6.71,26.0614,119.3061,CN,1612719056
2,bluff,54.05,85,100,5.10,-46.6000,168.3333,NZ,1612719275
3,lebu,64.33,73,51,8.50,-37.6167,-73.6500,CL,1612719065
4,waipawa,62.60,88,75,17.27,-41.4122,175.5153,NZ,1612719352


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [51]:
#  Get the indices of cities that have humidity over 100%.
humidity_scale = weather_data["Humidity"]

#create list loop
for humidity in humidity_scale:
    if humidity in humidity_scale > 100:
        print(f"{city['city']} is sweaty")
    else:
        Print("The weather is nice")






ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression